In [0]:
from pyspark.sql import functions as F, Window
from pyspark.sql.types import *

In [0]:
citrus_trade = spark.read.parquet("s3://pprachan-eraneos-data/trhs_citrus_parquet")

In [0]:
display(citrus_trade.limit(10))

DECLARANT,DECLARANT_ISO,PARTNER,PARTNER_ISO,PRODUCT_HS,FLOW,TRANSPORT_MODE,PERIOD,VALUE_IN_EUROS,QUANTITY_IN_KG,HS6,HS6_NAME,HS4,HS4_NAME
1,FR,6,GB,080510,1,1,202052,23084,1446,080510,Fresh or dried oranges,0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080510,1,3,202052,381874,834716,080510,Fresh or dried oranges,0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080510,1,5,202052,67,69,080510,Fresh or dried oranges,0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080510,2,1,202052,118537,107150,080510,Fresh or dried oranges,0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080510,2,2,202052,224,214,080510,Fresh or dried oranges,0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080510,2,3,202052,935325,880724,080510,Fresh or dried oranges,0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080521,1,1,202052,42842,91667,080521,Fresh or dried mandarins incl. tangerines and satsumas (excl. clementines),0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080521,2,1,202052,1015,470,080521,Fresh or dried mandarins incl. tangerines and satsumas (excl. clementines),0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080521,2,3,202052,362304,191311,080521,Fresh or dried mandarins incl. tangerines and satsumas (excl. clementines),0805,"Citrus fruit, fresh or dried"
1,FR,6,GB,080522,1,3,202052,1153,828,080522,Fresh or dried clementines incl. monreales,0805,"Citrus fruit, fresh or dried"


In [0]:
display(
    citrus_trade
        .filter(
            (F.col("DECLARANT_ISO") == "ES") &
            (F.col("FLOW")==2)
        )
        .withColumn("YEAR", F.substring("PERIOD",1,4))
        .groupBy("DECLARANT_ISO","HS6_NAME")
        .agg(
            F.sum("VALUE_IN_EUROS").alias("total_value"),
            F.sum("QUANTITY_IN_KG").alias("total_quantity"),
            F.when(F.sum("QUANTITY_IN_KG") != 0, F.sum("VALUE_IN_EUROS")/F.sum("QUANTITY_IN_KG")).otherwise(None).alias("avg_price")
        )
)

DECLARANT_ISO,HS6_NAME,total_value,total_quantity,avg_price
ES,Fresh or dried mandarins incl. tangerines and satsumas (excl. clementines),65635714,62050284,1.0577826525338707
ES,Fresh or dried clementines incl. monreales,302259096,282441901,1.0701637927298897
ES,"Fresh or dried citrus fruit (excl. oranges, lemons ""Citrus limon, Citrus limonum"", limes ""Citrus aurantifolia, Citrus latifolia"", grapefruit, mandarins, incl. tangerines and satsumas, clementines, wilkings and similar citrus hybrids)",1859860,1556557,1.1948550550991708
ES,Fresh or dried grapefruit,14472622,16195566,0.8936163144900277
ES,Fresh or dried wilkings and similar citrus hybrids,173698912,130881342,1.3271480055575835
ES,"Fresh or dried lemons ""Citrus limon, Citrus limonum"" and limes ""Citrus aurantifolia, Citrus latifolia""",284728645,246100528,1.1569607237900765
ES,Fresh or dried oranges,714519412,860234998,0.8306095586220267
ES,"Fresh or dried mandarins incl. tangerines and satsumas, clementines, wilkings and similar citrus hybrids",100834492,99858439,1.0097743666912318


In [0]:
display(citrus_trade.select("HS6_NAME","HS6").distinct())

HS6_NAME,HS6
Fresh or dried oranges,080510
"Fresh or dried citrus fruit (excl. oranges, lemons ""Citrus limon, Citrus limonum"", limes ""Citrus aurantifolia, Citrus latifolia"", grapefruit, mandarins, incl. tangerines and satsumas, clementines, wilkings and similar citrus hybrids)",080590
Fresh or dried mandarins incl. tangerines and satsumas (excl. clementines),080521
Fresh or dried grapefruit,080540
"Fresh or dried lemons ""Citrus limon, Citrus limonum"" and limes ""Citrus aurantifolia, Citrus latifolia""",080550
Fresh or dried clementines incl. monreales,080522
Fresh or dried wilkings and similar citrus hybrids,080529
"Fresh or dried mandarins incl. tangerines and satsumas, clementines, wilkings and similar citrus hybrids",080520
